In [32]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import spacy as sp
import en_core_web_sm
import string

from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from xgboost.sklearn import XGBClassifier 
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

In [2]:
rdata_json = pd.read_json('../relevantNewsNLTK4.json')
irdata_json = pd.read_json('../irrelevantNewsNLTK4.json')

In [3]:
rdata_json['relevance']=1
irdata_json['relevance']=0

In [4]:
data = rdata_json.append(irdata_json, ignore_index=True)
data.head()

,content,headline,source,summary,uid,index,label,relevance
0,lowly milkshake weapon choice britons determin...,Milkshakes become weapon of choice in UK Europ...,Agence France Presse,Former UK Independence Party leader Nigel Fara...,a437ff48-104a-54bb-bff7-c7a736158524,0,1,1
1,anz race set bring biggest white collar job lo...,ANZ's first assault in the looming job armageddon,News Ltd.,ANZ has moved to the front of the race that’ s...,366c92af-8143-5ffa-8702-4f26bd22c8b6,1,1,1
2,jul 10 carnival cruise stateroom attendants ex...,Carnival Cruise Line to collect your used soap...,Tribune Content Agency,Jul. 10-- Carnival Cruise Line stateroom atten...,863096d4-48f0-5a7c-bee6-384a76d575ee,2,1,1
3,chennai rohit 543 saravanan 546 wickets standa...,Standard CC bags fourth title in a row [New In...,SyndiGate Media Inc.,CHENNAI: R Rohit and P Saravanan took five wic...,3e4d6490-4224-595e-be26-4cb249209b8f,3,1,1
4,donald trump nominee lead fish wildlife servic...,Revealed: Trump's Wildlife Service pick has ti...,Guardian,New revelations show she also has ties to the ...,9f3e248d-b040-5058-bdc9-61c4de59f02a,4,1,1


In [5]:
# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = en_core_web_sm.load()
stop_words = sp.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [6]:
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [7]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [8]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))

In [10]:
X = data['content'] # the features we want to analyze
ylabels = data['relevance'] # the labels, or answers, we want to test against

# X_train and y_train are the entire dataset (for now)
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2,random_state = 42)

In [30]:

## Use this for MultinomialNB (Naive Bayes) Classifier in the pipeline after vectorizer
class DenseTransformer(TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()

In [37]:
# Classifier
# Note: classifier is a placeholder for now

classifier =XGBClassifier()

#('to_dense', DenseTransformer()),
# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x000002100F8402E8>),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=No...
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
                               min_child_weight=1, missing=None,
                               n_estimato

In [38]:
# Predicting with a test dataset
# Note: this will fail if testing data size is 0
predicted = pipe.predict(X_test)

# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Precision:",metrics.precision_score(y_test, predicted))
print("Recall:",metrics.recall_score(y_test, predicted))

Accuracy: 0.73775
Precision: 0.7411944869831547
Recall: 0.728184553660983


In [40]:
import pickle
pickle.dump(classifier, open('best_bow_xgboost_model.sav','wb'))